In [7]:
import os
import re
import pickle
import sys
import time
from glob import glob
from math import ceil
from urllib import request
from zipfile import ZipFile
import tarfile


import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import tensorflow as tf
import imageio

PIXEL_ROAD = 7
PIXEL_CAR = 10
PIXEL_MAX = 255

In [8]:
# Matplotlib configuration
%matplotlib inline

In [9]:
def augment_brightness_camera_images(image):
  # Randomly increase or decrease brightness, not used
  image = colors.rgb_to_hsv(image).astype(float)
  random_bright = .5 + np.random.uniform()
  image[:, :, 2] = image[:, :, 2] * random_bright
  image[:, :, 2][image[:, :, 2] > PIXEL_MAX] = PIXEL_MAX
  image = colors.hsv_to_rgb(image).astype(int)
  return image

def rotate(image, label, angle):
  # Rotate by given angle
  angle = np.random.random() * angle * 2 - angle
  image = scipy.misc.imrotate(image, angle)
  label = scipy.misc.imrotate(label, angle)
  return image, label

def flip(image, label):
  # Flip images horizontally
  image = image[:, ::-1, :]
  label = label[:, ::-1, :]
  return image, label

def randomize(image, label):
  # Randomize a single image
  image = augment_brightness_camera_images(image)
  image, label = rotate(image, label, angle=5)
  if np.random.random() < .5:
    image, label = flip(image, label)
  return image, label


def gen_batch(feature_img_paths, image_shape, augment=False):
  # Returns a batch generator for given images

  def get_batches(batch_size, shuffle=False):
    #feature_img_paths = glob(os.path.join(data_dir, 'CameraRGB', '*.png'))
    if shuffle:
      np.random.shuffle(feature_img_paths)
    foreground_road = np.array([PIXEL_ROAD, 0, 0])
    foreground_car = np.array([PIXEL_CAR, 0, 0])
    for i in range(0, len(feature_img_paths), batch_size):
      features, labels = [], []
      for fn in feature_img_paths[i:i + batch_size]:
        # Preprocess features
        feature = imageio.imread(fn)
        #feature = scipy.misc.imread(fn)
        
        feature = scipy.misc.imresize(feature, image_shape) # If scipy is used, image_shape = (160, 576)
        #feature = cv2.resize(feature, image_shape) # If cv2 is used, image_shape = (576, 160)
        #feature = transform.resize(feature, image_shape)
        
        
        # Load labels
        label = imageio.imread(fn.replace('CameraRGB', 'CameraSeg'))
        #label = scipy.misc.imread(fn.replace('CameraRGB', 'CameraSeg'))
        
        #***************************************************************
        ##The following steps marks the HOOD pixels as Other(0)
        a = label[IMAGE_BOTTOM_HOOD:,:,:]==10
        b=np.copy(label[IMAGE_BOTTOM_HOOD:,:,:])
        b[a]=0
        label=np.concatenate((label[:IMAGE_BOTTOM_HOOD,:,:],b),axis=0)
        #***************************************************************
        
        #***************************************************************
        #The following step marks the lane marks as roads
        label[label==6]=7
        #***************************************************************
        
        label = scipy.misc.imresize(label, image_shape)
        #label = cv2.resize(label, image_shape)
        
        if augment:
          feature, label = randomize(feature, label)

        # Convert label image to output array
        gt_fg_road = np.all(label == foreground_road, axis=2)
        gt_fg_car = np.all(label == foreground_car, axis=2)
        gt_road_and_car = gt_fg_road | gt_fg_car
            
        gt_fg_road = gt_fg_road.reshape(*gt_fg_road.shape, 1)
        gt_fg_car = gt_fg_car.reshape(*gt_fg_car.shape, 1)
        gt_road_and_car = gt_road_and_car.reshape(*gt_road_and_car.shape, 1)
            
        label = np.concatenate((gt_fg_road,gt_fg_car,np.invert(gt_road_and_car)), axis=2)
        
        features.append(feature)
        labels.append(label)
      yield np.array(features), np.array(labels)

  return get_batches

In [10]:
data_dir = '../lyft_data/Train'
data_dir1 = '../lyft_data/selected1'
data_dir2 = '../lyft_data/selected2'
data_dir3 = '../lyft_data/selected3_episode0'
data_dir4 = '../lyft_data/selected3_episode1'
data_dir5 = '../lyft_data/selected3_episode2'
data_dir6 = '../lyft_data/selected3_episode4'
data_dir7 = '../lyft_data/selected3_episode13'

image_shape = (160, 576)
#image_shape = (576, 160)

#### Training

In [11]:
num_classes = 3
# Hyperparameters
augment_training = True
batch_size = 8
epochs = 50
min_epochs = 5
patience = 2
random_seed = 42
starting_learning_rate = 1e-4



loaded_graph = tf.Graph()
load_dir = './model_combinedv4.ckpt'

with tf.Session(graph=loaded_graph) as sess:
    #Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)
    
    
    # Gather required tensor references
    input_image = loaded_graph.get_tensor_by_name('image_input:0')
    correct_label = loaded_graph.get_tensor_by_name('correct_label:0')
    learning_rate = loaded_graph.get_tensor_by_name('lr:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    predicted = loaded_graph.get_tensor_by_name('predicts:0')
    cross_entropy_loss = loaded_graph.get_tensor_by_name('loss:0')
    
    global_step = loaded_graph.get_tensor_by_name('global_step:0')
    
    train_op = loaded_graph.get_tensor_by_name('train_adam:0')
    
    # Prepare training and test datasets
    filenames = glob(os.path.join(data_dir, 'CameraRGB', '*.png'))
    filenames.extend(glob(os.path.join(data_dir1, 'CameraRGB', '*.png')))
    filenames.extend(glob(os.path.join(data_dir2, 'CameraRGB', '*.png')))
    filenames.extend(glob(os.path.join(data_dir3, 'CameraRGB', '*.png')))
    filenames.extend(glob(os.path.join(data_dir4, 'CameraRGB', '*.png')))
    filenames.extend(glob(os.path.join(data_dir5, 'CameraRGB', '*.png')))
    filenames.extend(glob(os.path.join(data_dir6, 'CameraRGB', '*.png')))
    filenames.extend(glob(os.path.join(data_dir7, 'CameraRGB', '*.png')))
    print('Dataset contains', len(filenames), 'examples.')
    np.random.seed(random_seed)  # Make it reproduceable
    np.random.shuffle(filenames)
    training = int(len(filenames) * 1.0)  # 85% for training, 15% for testing
    training, testing = filenames[:training], filenames[training:]
    
    print('Training on', len(training), 'samples.')
    
    get_batches_fn = gen_batch(
          feature_img_paths=training,
          image_shape=image_shape,
          augment=augment_training)
    
    # Continue training while the loss improves
    best_loss = 1e9
    failure = 0
    for e in range(epochs):
        epoch_loss = 0
        num_images = 0
        sys.stdout.flush()
        
        for images, labels in tqdm(get_batches_fn(batch_size, shuffle=True),
                                   desc='Training epoch {}'.format(e + 1),
                                   total=ceil(len(training) / batch_size)):
            step, _, loss = sess.run([
              global_step,
              train_op,
              cross_entropy_loss], feed_dict={
                  input_image: images,
                  correct_label: labels,
                  keep_prob: 0.75,
                  learning_rate: starting_learning_rate})
            writer.add_summary(summary, step)
            epoch_loss += loss * len(images)
            
            num_images += len(images)

        epoch_loss /= num_images
        sys.stderr.flush()
        print('Epoch {} loss: {:.3f}'.format(e + 1, epoch_loss))
        print(num_images)
        if e >= min_epochs and epoch_loss > best_loss:
          if failure == patience:
            break
          failure += 1
        else:
            failure = 0
            best_loss = epoch_loss
            #if os.path.exists('./model_combinedv4.ckpt.meta'):
            print('Saving model without metadata')
            saver.save(sess, './model_combinedv4.ckpt',write_meta_graph=False)
            #print('Saving model')
            #saver.save(sess, './model_combinedv4.ckpt')
print("Training Done")



INFO:tensorflow:Restoring parameters from ./model_combinedv4.ckpt


ValueError: Duplicate node name in graph: 'conv1_1/filter/Adam'